#### Data Class

In [1]:
import random

class Sentiment:
    NEGATIVE = 'NEGATIVE'
    POSITIVE = 'POSITIVE'
    NEUTRAL = 'NEUTRAL'
class Review:
    def __init__(self, text, score):
        self.text = text 
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: 
            return Sentiment.POSITIVE

class ReviewContainer: 
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def evenly_distribution(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

#### Load data 

In [3]:
import json 

file_name = './Books_small_10000.json'

reviews = []
with open(file_name) as file:
    for line in file:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

#### Prep Data 

In [4]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)




In [5]:
train_container.evenly_distribution()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribution()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


#### Bags of words vectoritzation 

In [6]:
from sklearn.feature_extraction.text import  TfidfVectorizer

In [7]:
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)


print(train_x[0])
print(train_x_vectors[0].toarray())

***received an arc in exchange for an honest review***I was excited to read this book after reading the first in the series Connected By The Sea. I was interested in reading Henry's story after being introduced to his character in Sydney's story. I was slightly disappointed to say the least. There were so many scenes that had me scratching my head in confusion. The idea that Henry was so in love with Sydney that he thought his life was over because she rejected him to fall completely and madly in love with Renee so quickly and profess more then once that he had no feelings what so ever for Sydney seemed a little far fetched to me. The fact the he was honest with Renee and told her about his feelings for someone else and her be okay with it and want to be with him anyway didn't seemPlausible...I kept thinking in my head what woman would do that. Some of the sex scenes had me skimming...especially the masterbation ones. I didn't understand why Henry would feel like a pervert for being at

## Classification

In [8]:
from sklearn import svm 

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])


array(['NEGATIVE'], dtype='<U8')

#### Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Evaluation 

In [11]:
print(clf_svm.score(test_x_vectors, test_y)) 
print(clf_dec.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

# the svm classification has the best result

0.8076923076923077
0.6274038461538461
0.8052884615384616


In [12]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average= None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])

# All models perform great on 'POSITIVE' the rest is shit...

array([0.80582524, 0.80952381])

## RESULT!

In [13]:
test_set = ['Tom is stupid']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE'], dtype='<U8')

#### Tuning the model with GridSerach

In [14]:
# lets tune the paramter of SVC to make our model more accurate

from sklearn.model_selection import GridSearchCV

paranters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, paranters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

#### Saving the model

In [15]:
import pickle 

with open('./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)


##### Load the model

In [16]:
with open('./models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [28]:
print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

This book was not a really good read, tried a new author but the book was alittle slow on the story line.


array(['NEGATIVE'], dtype='<U8')

## Test the algorithm on a new data set

#### Load the data
Go to kaggle.com and search for Amazon clothing data...

In [18]:
import pandas as pd 
data = pd.read_csv('/WomensClothingE-CommerceReviews.csv')

In [19]:
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


#### Clean the data

In [20]:
# get rid of all columns and only use the review text column!

df = data['Review Text'].to_frame()


In [21]:
# get rif of all empty rows!
df.isnull().sum()

# 845 are null delete them!
df = df.dropna(how='any',axis=0) 

# print your resutls!
df.isnull().sum()

Review Text    0
dtype: int64

In [ ]:
# Convert the column into a list 
import numpy as np

new_list = df['Review Text'].tolist()

print(new_list)

#### Test the algorithm with the data set!

In [23]:
test_set = new_list
new_test = vectorizer.transform(test_set)

feedback = clf_svm.predict(new_test)

In [24]:
# create new dataframe

d = {'Reviews': new_list, 'Feedback': feedback}
data = pd.DataFrame(d)

In [ ]:
print(data.head(10))

In [26]:
data.to_csv('results.csv', index = False)